# Music-Recommender System
##**General outline:**

###Hack To Get Data
*   Before using, adjust this file to your file hierachy under **Hack to get Data/Local storage**
*   The msd subset is stored as hfd5 files, **HDF5 Getters (imported) and necessary function** contains all functions to process those
*   Two individual datasets are created

> * one non-audio set (containing features like artist etc.)
>> * **Extract non audio**
> * one audio set (containing musical features)
>> * **Extract audio**
* Since already the subset takes some time to be processed. 'smallset' is a subset of the subset containing 53 files. Whenever the hfd5 files are extracted, use *smallset* instead of *msd_subset_data_path* to extract if you want quick results
* Pandas Dataframe for non Audio: *nonAudioData*
* Pandas Dataframe for  Audio: *audioData*






# New Section

In [0]:
!pip install tables

##Hack to get the data 

In [0]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import tarfile
import tables
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imshow
from google.colab import drive
from collections import Counter
from itertools import repeat, chain
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist


**Local Storage**

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
msd_subset_path=r'/content/drive/My Drive/IR/Dataprocessing/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
smallset = r'/content/drive/My Drive/IR/Dataprocessing/A'
msd_code_path=r'/content/drive/My Drive/IR/Dataprocessing'
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

#Is the given path really a path?
print(os.path.isdir(msd_subset_path))
print(os.path.exists(msd_subset_path))
print(os.path.isdir(msd_code_path))
print(os.path.exists(msd_code_path))
print(os.path.isdir(smallset))

True
True
True
True
True


###HDF5 Getters (imported) and necessary function

In [0]:
"""
Thierry Bertin-Mahieux (2010) Columbia University
tb2332@columbia.edu
This code contains a set of getters functions to access the fields
from an HDF5 song file (regular file with one song or
aggregate / summary file with many songs)
This is part of the Million Song Dataset project from
LabROSA (Columbia University) and The Echo Nest.
Copyright 2010, Thierry Bertin-Mahieux
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""


def open_h5_file_read(h5filename):
    """
    Open an existing H5 in read mode.
    Same function as in hdf5_utils, here so we avoid one import
    """
    return tables.open_file(h5filename, mode='r')


def get_num_songs(h5):
    """
    Return the number of songs contained in this h5 file, i.e. the number of rows
    for all basic informations like name, artist, ...
    """
    return h5.root.metadata.songs.nrows

def get_artist_familiarity(h5,songidx=0):
    """
    Get artist familiarity from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_familiarity[songidx]

def get_artist_hotttnesss(h5,songidx=0):
    """
    Get artist hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_hotttnesss[songidx]

def get_artist_id(h5,songidx=0):
    """
    Get artist id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_id[songidx]

def get_artist_mbid(h5,songidx=0):
    """
    Get artist musibrainz id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_mbid[songidx]

def get_artist_playmeid(h5,songidx=0):
    """
    Get artist playme id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_playmeid[songidx]

def get_artist_7digitalid(h5,songidx=0):
    """
    Get artist 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_7digitalid[songidx]

def get_artist_latitude(h5,songidx=0):
    """
    Get artist latitude from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_latitude[songidx]

def get_artist_longitude(h5,songidx=0):
    """
    Get artist longitude from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_longitude[songidx]

def get_artist_location(h5,songidx=0):
    """
    Get artist location from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_location[songidx]

def get_artist_name(h5,songidx=0):
    """
    Get artist name from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.artist_name[songidx]

def get_release(h5,songidx=0):
    """
    Get release from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.release[songidx]

def get_release_7digitalid(h5,songidx=0):
    """
    Get release 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.release_7digitalid[songidx]

def get_song_id(h5,songidx=0):
    """
    Get song id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.song_id[songidx]

def get_song_hotttnesss(h5,songidx=0):
    """
    Get song hotttnesss from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.song_hotttnesss[songidx]

def get_title(h5,songidx=0):
    """
    Get title from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.title[songidx]

def get_track_7digitalid(h5,songidx=0):
    """
    Get track 7digital id from a HDF5 song file, by default the first song in it
    """
    return h5.root.metadata.songs.cols.track_7digitalid[songidx]

def get_similar_artists(h5,songidx=0):
    """
    Get similar artists array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:]
    return h5.root.metadata.similar_artists[h5.root.metadata.songs.cols.idx_similar_artists[songidx]:
                                            h5.root.metadata.songs.cols.idx_similar_artists[songidx+1]]

def get_artist_terms(h5,songidx=0):
    """
    Get artist terms array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                            h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_freq(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_freq[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                              h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_artist_terms_weight(h5,songidx=0):
    """
    Get artist terms array frequencies. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.metadata.songs.nrows == songidx + 1:
        return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:]
    return h5.root.metadata.artist_terms_weight[h5.root.metadata.songs.cols.idx_artist_terms[songidx]:
                                                h5.root.metadata.songs.cols.idx_artist_terms[songidx+1]]

def get_analysis_sample_rate(h5,songidx=0):
    """
    Get analysis sample rate from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.analysis_sample_rate[songidx]

def get_audio_md5(h5,songidx=0):
    """
    Get audio MD5 from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.audio_md5[songidx]

def get_danceability(h5,songidx=0):
    """
    Get danceability from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.danceability[songidx]

def get_duration(h5,songidx=0):
    """
    Get duration from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.duration[songidx]

def get_end_of_fade_in(h5,songidx=0):
    """
    Get end of fade in from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.end_of_fade_in[songidx]

def get_energy(h5,songidx=0):
    """
    Get energy from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.energy[songidx]

def get_key(h5,songidx=0):
    """
    Get key from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.key[songidx]

def get_key_confidence(h5,songidx=0):
    """
    Get key confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.key_confidence[songidx]

def get_loudness(h5,songidx=0):
    """
    Get loudness from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.loudness[songidx]

def get_mode(h5,songidx=0):
    """
    Get mode from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.mode[songidx]

def get_mode_confidence(h5,songidx=0):
    """
    Get mode confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.mode_confidence[songidx]

def get_start_of_fade_out(h5,songidx=0):
    """
    Get start of fade out from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.start_of_fade_out[songidx]

def get_tempo(h5,songidx=0):
    """
    Get tempo from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.tempo[songidx]

def get_time_signature(h5,songidx=0):
    """
    Get signature from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.time_signature[songidx]

def get_time_signature_confidence(h5,songidx=0):
    """
    Get signature confidence from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.time_signature_confidence[songidx]

def get_track_id(h5,songidx=0):
    """
    Get track id from a HDF5 song file, by default the first song in it
    """
    return h5.root.analysis.songs.cols.track_id[songidx]

def get_segments_start(h5,songidx=0):
    """
    Get segments start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    h5 = os.path.normpath(h5)
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_start[h5.root.analysis.songs.cols.idx_segments_start[songidx]:]
    return h5.root.analysis.segments_start[h5.root.analysis.songs.cols.idx_segments_start[songidx]:
                                           h5.root.analysis.songs.cols.idx_segments_start[songidx+1]]
    
def get_segments_confidence(h5,songidx=0):
    """
    Get segments confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_confidence[h5.root.analysis.songs.cols.idx_segments_confidence[songidx]:]
    return h5.root.analysis.segments_confidence[h5.root.analysis.songs.cols.idx_segments_confidence[songidx]:
                                                h5.root.analysis.songs.cols.idx_segments_confidence[songidx+1]]

def get_segments_pitches(h5,songidx=0):
    """
    Get segments pitches array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_pitches[h5.root.analysis.songs.cols.idx_segments_pitches[songidx]:,:]
    return h5.root.analysis.segments_pitches[h5.root.analysis.songs.cols.idx_segments_pitches[songidx]:
                                             h5.root.analysis.songs.cols.idx_segments_pitches[songidx+1],:]

def get_segments_timbre(h5,songidx=0):
    """
    Get segments timbre array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:,:]
    return h5.root.analysis.segments_timbre[h5.root.analysis.songs.cols.idx_segments_timbre[songidx]:
                                            h5.root.analysis.songs.cols.idx_segments_timbre[songidx+1],:]

def get_segments_loudness_max(h5,songidx=0):
    """
    Get segments loudness max array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_max[h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx]:]
    return h5.root.analysis.segments_loudness_max[h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx]:
                                                  h5.root.analysis.songs.cols.idx_segments_loudness_max[songidx+1]]

def get_segments_loudness_max_time(h5,songidx=0):
    """
    Get segments loudness max time array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_max_time[h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx]:]
    return h5.root.analysis.segments_loudness_max_time[h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx]:
                                                       h5.root.analysis.songs.cols.idx_segments_loudness_max_time[songidx+1]]

def get_segments_loudness_start(h5,songidx=0):
    """
    Get segments loudness start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.segments_loudness_start[h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx]:]
    return h5.root.analysis.segments_loudness_start[h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx]:
                                                    h5.root.analysis.songs.cols.idx_segments_loudness_start[songidx+1]]

def get_sections_start(h5,songidx=0):
    """
    Get sections start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.sections_start[h5.root.analysis.songs.cols.idx_sections_start[songidx]:]
    return h5.root.analysis.sections_start[h5.root.analysis.songs.cols.idx_sections_start[songidx]:
                                           h5.root.analysis.songs.cols.idx_sections_start[songidx+1]]

def get_sections_confidence(h5,songidx=0):
    """
    Get sections confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.sections_confidence[h5.root.analysis.songs.cols.idx_sections_confidence[songidx]:]
    return h5.root.analysis.sections_confidence[h5.root.analysis.songs.cols.idx_sections_confidence[songidx]:
                                                h5.root.analysis.songs.cols.idx_sections_confidence[songidx+1]]

def get_beats_start(h5,songidx=0):
    """
    Get beats start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.beats_start[h5.root.analysis.songs.cols.idx_beats_start[songidx]:]
    return h5.root.analysis.beats_start[h5.root.analysis.songs.cols.idx_beats_start[songidx]:
                                        h5.root.analysis.songs.cols.idx_beats_start[songidx+1]]

def get_beats_confidence(h5,songidx=0):
    """
    Get beats confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.beats_confidence[h5.root.analysis.songs.cols.idx_beats_confidence[songidx]:]
    return h5.root.analysis.beats_confidence[h5.root.analysis.songs.cols.idx_beats_confidence[songidx]:
                                             h5.root.analysis.songs.cols.idx_beats_confidence[songidx+1]]

def get_bars_start(h5,songidx=0):
    """
    Get bars start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.bars_start[h5.root.analysis.songs.cols.idx_bars_start[songidx]:]
    return h5.root.analysis.bars_start[h5.root.analysis.songs.cols.idx_bars_start[songidx]:
                                       h5.root.analysis.songs.cols.idx_bars_start[songidx+1]]

def get_bars_confidence(h5,songidx=0):
    """
    Get bars start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.bars_confidence[h5.root.analysis.songs.cols.idx_bars_confidence[songidx]:]
    return h5.root.analysis.bars_confidence[h5.root.analysis.songs.cols.idx_bars_confidence[songidx]:
                                            h5.root.analysis.songs.cols.idx_bars_confidence[songidx+1]]

def get_tatums_start(h5,songidx=0):
    """
    Get tatums start array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.tatums_start[h5.root.analysis.songs.cols.idx_tatums_start[songidx]:]
    return h5.root.analysis.tatums_start[h5.root.analysis.songs.cols.idx_tatums_start[songidx]:
                                         h5.root.analysis.songs.cols.idx_tatums_start[songidx+1]]

def get_tatums_confidence(h5,songidx=0):
    """
    Get tatums confidence array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.analysis.songs.nrows == songidx + 1:
        return h5.root.analysis.tatums_confidence[h5.root.analysis.songs.cols.idx_tatums_confidence[songidx]:]
    return h5.root.analysis.tatums_confidence[h5.root.analysis.songs.cols.idx_tatums_confidence[songidx]:
                                              h5.root.analysis.songs.cols.idx_tatums_confidence[songidx+1]]

def get_artist_mbtags(h5,songidx=0):
    """
    Get artist musicbrainz tag array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.musicbrainz.songs.nrows == songidx + 1:
        return h5.root.musicbrainz.artist_mbtags[h5.root.musicbrainz.songs.cols.idx_artist_mbtags[songidx]:]
    return h5.root.musicbrainz.artist_mbtags[h5.root.metadata.songs.cols.idx_artist_mbtags[songidx]:
                                             h5.root.metadata.songs.cols.idx_artist_mbtags[songidx+1]]

def get_artist_mbtags_count(h5,songidx=0):
    """
    Get artist musicbrainz tag count array. Takes care of the proper indexing if we are in aggregate
    file. By default, return the array for the first song in the h5 file.
    To get a regular numpy ndarray, cast the result to: numpy.array( )
    """
    if h5.root.musicbrainz.songs.nrows == songidx + 1:
        return h5.root.musicbrainz.artist_mbtags_count[h5.root.musicbrainz.songs.cols.idx_artist_mbtags[songidx]:]
    return h5.root.musicbrainz.artist_mbtags_count[h5.root.metadata.songs.cols.idx_artist_mbtags[songidx]:
                                                   h5.root.metadata.songs.cols.idx_artist_mbtags[songidx+1]]

def get_year(h5,songidx=0):
    """
    Get release year from a HDF5 song file, by default the first song in it
    """
    return h5.root.musicbrainz.songs.cols.year[songidx]
 


In [0]:
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [0]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

### Extract non Audio

In [0]:
all_song_id = []
all_artist_id=[]
all_artist_name=[]
all_release=[]
all_title=[]
all_danceability=[]
all_track_id=[]
all_year=[]
all_similar_artists=[]
all_artist_terms=[]
all_artist_terms_freq=[]
all_artist_terms_weight =[]

def nonAudio(filename):
    h5 = open_h5_file_read(filename)
    tmp = get_song_id(h5)
    all_song_id.append(tmp)
    tmp = get_artist_id(h5)
    all_artist_id.append(tmp)
    tmp = get_artist_name(h5)
    all_artist_name.append(tmp)
    tmp = get_release(h5)
    all_release.append(tmp)
    tmp = get_title(h5)
    all_title.append(tmp)
    tmp = get_danceability(h5)
    all_danceability.append(tmp)
    tmp = get_track_id(h5)
    all_track_id.append(tmp)
    tmp = get_year(h5)
    all_year.append(tmp)
    tmp = get_similar_artists(h5)
    all_similar_artists.append(tmp)
    tmp = get_artist_terms(h5)
    all_artist_terms.append(tmp)
    tmp = get_artist_terms_freq(h5)
    all_artist_terms_freq.append(tmp)
    tmp = get_artist_terms_weight(h5)
    all_artist_terms_weight.append(tmp)
    h5.close()

In [0]:
count1 = 0
count2 = 0
for i in all_danceability:
    if i == 0:
        count1+=1
    else:
        count2+=1
    
print(count1,count2)

0 0


Change *smallset* to* msd_subset_data_path* to apply function to whole subset $\rightarrow$ will take ages

In [0]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=nonAudio)
t2 = time.time()
print(t2-t1)

3644.0423991680145


In [0]:
nonAudio = [all_artist_id,
all_artist_name,
all_release,
all_title,
all_danceability,
all_track_id,
all_year,
all_similar_artists,
all_artist_terms,
all_artist_terms_freq,
all_artist_terms_weight]

nonAudioLabels = ['artist_id',
'artist_name',
'release',
'title',
'danceability',
'track_id',
'year',
'similar_artists',
'artist_terms',
'artist_terms_freq',
'artist_terms_weight']

dataNonAudio = pd.DataFrame(nonAudio,index = nonAudioLabels, columns = all_song_id)
nonAudioData = dataNonAudio.transpose()
#nonAudioData = nonAudioData[~nonAudioData.index.duplicated(keep='first')]

In [0]:
nonAudioData.head()

,artist_id,artist_name,release,title,danceability,track_id,year,similar_artists,artist_terms,artist_terms_freq,artist_terms_weight
b'SOCIWDW12A8C13D406',b'ARMJAGH1187FB546F3',b'The Box Tops',b'Dimensions',b'Soul Deep',0,b'TRAAABD128F429CF47',1969,"[b'ARSZWK21187B9B26D7', b'ARLDW2Y1187B9B544F',...","[b'blue-eyed soul', b'pop rock', b'blues-rock'...","[1.0, 0.8931999928346939, 0.7860602885494408, ...","[1.0, 0.8459884034332037, 0.8306895698215381, ..."
b'SOMJBYD12A6D4F8557',b'ARD0S291187B9B7BF5',b'Rated R',b'Da Ghetto Psychic',b'Keepin It Real (Skit)',0,b'TRAAAMQ128F1460CD3',0,"[b'ARF93II1187B99F981', b'ART6ONC11C8A421DB9',...","[b'breakbeat', b'dirty south rap', b'hip hop',...","[1.0, 0.8386187613378114, 0.9353130749575775, ...","[1.0, 0.9258286857243918, 0.8650794411328923, ..."
b'SONHOTT12A8C13493C',b'AR7G5I41187FB4CE6C',b'Adam Ant',b'Friend Or Foe',b'Something Girls',0,b'TRAAAEF128F4273421',1982,"[b'AR4R0741187FB39AF2', b'AR0D7K21187B9AD14E',...","[b'pop rock', b'new wave', b'dance rock', b'ro...","[0.9885838625154639, 0.9672504640243684, 0.820...","[1.0, 0.9636972066614938, 0.9267729972686404, ..."
b'SOXVLOJ12AB0189215',b'ARKRRTF1187B9984DA',b'Sonora Santanera',b'Las Numero 1 De La Sonora Santanera',b'Amor De Cabaret',0,b'TRAAADZ128F9348C2E',0,"[b'ARFSJUG11C8A421AAD', b'AR8SD041187FB36015',...","[b'salsa', b'cumbia', b'tejano', b'ranchera', ...","[1.0, 0.9422390717941641, 0.9422390717941641, ...","[1.0, 0.9582578450180738, 0.9582578450180738, ..."
b'SOMZWCG12A8C13C480',b'ARD7TVE1187B99BFB1',b'Casual',b'Fear Itself',"b""I Didn't Mean To""",0,b'TRAAAAW128F429D538',0,"[b'ARV4KO21187FB38008', b'ARWHM281187FB3D381',...","[b'hip hop', b'underground rap', b'g funk', b'...","[1.0, 0.7761362332679642, 0.7296697949672141, ...","[1.0, 0.8979359555142553, 0.8842618474718359, ..."


###Extract Audio

In [0]:
all_analysis_sample_rate = []
all_tempo = []
all_time_signature = []
all_segments_pitches= []
all_segments_timbre= []
all_segments_loudness_max= []
all_song_id = []

def extractAudio(filename): 
  
    h5 = open_h5_file_read(filename)
    tmp = get_song_id(h5)
    all_song_id.append(tmp)
    tmp = get_analysis_sample_rate(h5)
    all_analysis_sample_rate.append(tmp)
    tmp = get_time_signature(h5)
    all_time_signature.append(tmp)
    tmp = get_tempo(h5)
    all_tempo.append(tmp)
    tmp = get_segments_pitches(h5)
    all_segments_pitches.append(tmp)
    tmp = get_segments_timbre(h5) 
    all_segments_timbre.append(tmp)
    tmp = get_segments_loudness_max(h5)
    all_segments_loudness_max.append(tmp)
    h5.close()    

run for a small subset

In [0]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=extractAudio)
t2 = time.time()

run for the whole subset

In [0]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=extractAudio)
t2 = time.time()

Extracting the features

In [0]:
Audio = [
    all_analysis_sample_rate,
    all_time_signature,
    all_tempo,
    all_segments_pitches,
    all_segments_timbre, #pure mfcc
    all_segments_loudness_max
]
AudioLabels = [
    'analysis_sample_rate',
    'time_signature',
    'tempo',
    'segments_pitches',
    'segments_timbre', #pure mfcc
    'segments_loudness_max'
]

In [0]:
dataAudio = pd.DataFrame(Audio,index = AudioLabels, columns = all_song_id)
audioData = dataAudio.transpose()

In [0]:
audioData.head()

NameError: ignored

## Get training and test indices


In [0]:
def get_prev(tag_list):
    """Returns the prevalent tag for a tag_list of a given song"""
    if len(tag_list)>0:
        prev = tag_list[0]
    else:
        prev = b'other'
    return prev


def add_prevalent_tag(df):
    """Adds the prevalent tag/artist_term as a column"""
    # Adds a column with the highest weighted tag
    df = df.assign(prevalent_tag=df.apply(lambda song: get_prev(song.loc['artist_terms']), axis=1))
    # Get array of all prevalent tags (all_tags)
    all_tags = df['prevalent_tag'].values
    i = 0
    for index, song in df.iterrows():
        # Delete prevalent term of that song from array
        all_tags[i] = None
        # Go through song_tags until you find a tag that already occurs for 
        # another song or end of song_tags
        song_tags = song['artist_terms']
        single = True
        j = 0
        while single and j<len(song_tags):
            new_tag = song_tags[j]
            j += 1
            if new_tag in all_tags:
                single = False
        if single:
            new_tag = b'other'
        df.iloc[i]['prevalent_tag'] = new_tag 
        all_tags[i] = new_tag
        i += 1
    return df

nonAudioData = add_prevalent_tag(nonAudioData)

In [0]:
def get_train_test_split(df, test_size=0.2, stratify=True):
    """Returns train and test split"""
    if stratify:
        split = train_test_split(df, test_size=test_size, shuffle=True, stratify=df['prevalent_tag'])
    else: 
        split = train_test_split(df, test_size=test_size, shuffle=True)
    return split

def save_train_test_split(data):
    """Saves the IDs for training and testing data in a .npy file.
    Stratifies according to the prevalent tag/genre"""
    df = data.copy()
    train, test = get_train_test_split(df, stratify=False)
    train_indices = list(train.index)
    test_indices = list(test.index)
    np.save('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TrainIndicesNew', train_indices)
    np.save('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TestIndicesNew', test_indices)

In [0]:
def get_test_data():
    # Load latents
    latents = np.load(msd_code_path+'/Latent features/latent_features1.npy')

    # Get test_data
    test_ids = np.load('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TestIndices.npy') 
    test_data = nonAudioData.loc[test_ids]

    # Eliminate duplicates if necessary
    duplicate_test_ids = test_data.index.get_duplicates() 
    duplicate_test_indices = np.array([np.where(test_data.index.get_loc(id))[0].tolist() for id in duplicate_test_ids])
    latents = np.delete(latents, duplicate_test_indices, axis=0)
    test_data = test_data.drop(duplicate_test_ids)
    test_ids = test_data.index

    # Insert latents as columns in test_df
    latent_df = pd.DataFrame(latents, index=test_ids)
    test_df = pd.concat([test_data, latent_df], axis=1)
    
    return test_df


In [0]:
#save_train_test_split(nonAudioData)

In [0]:
#train_ids = np.load('/content/drive/My Drive/IR/Dataprocessing/TrainTestSplit/TrainIndices.npy')
#train_data = nonAudioData.loc[train_ids]

## Get user profile
Use **get_all_tags(nonAudioData)** to get all possible tags  
Use **get_genre_playlist(nonAudioData, tag, max_length)** to get a playlist with desired *max_length* sorted by tag_weight for given *tag*  
Use **get_profile(nonAudioData, length, min_nb_genres)** to get a DataFrame of *length* songs from at least *min_nb_genres* different genres

In [0]:
def get_all_tags(df):
    """Returns a list of all tags/artist_terms"""
    return list(set(np.concatenate(df['artist_terms'].values)))


def get_common_tags(df):
    """Returns the 100 most common tags/artist_terms"""
    tags = df.loc[:,'prevalent_tag'].values
    sorted_tags = list(chain.from_iterable(repeat(i, c) for i,c in Counter(tags).most_common()))
    prev_tag = None
    unique_sorted_tags = []
    for tag in sorted_tags:
        if prev_tag != tag:
            unique_sorted_tags.append(tag)
            prev_tag = tag
            
    print(unique_sorted_tags[-20:])
    return unique_sorted_tags[:100]

def get_songs_with_tag(df, tag):
    """Returns a DataFrame containing all songs that were tagged with a given
    tag/artist_term.
    example use: get_songs_with_tag(nonAudioData, b'jazz')"""
    mask = []
    for tags in df['artist_terms'].values:
        if tag in tags:
            mask.append(True)
        else:
            mask.append(False)
    return df[mask]

def get_tag_weight(song, tag):
    """Returns the tag_weight/artist_term_weight for a given song and a given tag/artist_term"""
    tags = song['artist_terms']
    tag_weights = song['artist_terms_weight']
    return tag_weights[list(tags).index(tag)]

def get_genre_playlist(df, tag, max_length=10):
    """Returns a playlist with desired length sorted by tag_weight/artist_term_weight for a 
    given tag/artist term"""
    # get all songs with the given tag
    playlist = get_songs_with_tag(df, tag)
    # create new column with weight of given tag
    playlist = playlist.assign(weight=playlist.apply(lambda x: get_tag_weight(x, tag), axis=1))
    # sort by tag weight
    playlist = playlist.sort_values(by="weight", ascending=False)
    # clip after max_length songs
    short_playlist = playlist.iloc[:max_length]
    return short_playlist

In [0]:
common_tags = get_common_tags(nonAudioData) #Computed only once to save time

def get_profile(df, length=100, min_nb_genres=5):
    """Returns a playlist with desired length with songs from at least 
    min_nb_genres random genres."""

    playlist = pd.DataFrame()
    playlist_tags = []
    nb_tags = len(common_tags)
    tags = common_tags.copy()
    
    while len(playlist)<length:
        # Get a random tag and remove it from tags (so it won't be reused)
        random_tag = random.choice(tags)
        playlist_tags.append(random_tag)
        tags.remove(random_tag)        
        # Get playlist for the random tag
        max_length = min(length - len(playlist), length//min_nb_genres)
        genre_playlist = get_genre_playlist(df, random_tag, max_length=max_length)
        # Add genre playlist to playlist
        playlist = playlist.append(genre_playlist)
    playlist = playlist.drop('weight', axis=1)
    return playlist, playlist_tags


## K-means clustering

In [0]:
def get_dist(centroid, latent):
    """Helper function to compute the distance between a cluster centroid and a latent variable"""
    X = np.transpose(np.stack((centroid, latent), axis=1))
    return pdist(X)[0]

In [0]:
def get_recommendations(all_songs, input_songs, nb_clusters, nb_rec_per_cluster=10):
    """Gives recommenations from all_songs based on input_songs
    args:
    all_songs: DataFrame containing all songs to choose from
    input_songs: Songs on which to base the user profile
    nb_clusters: Number of clusters to fit to the profile. Optimally the number
                 of tags the profile has been created from
    nb_rec_per_cluster: Number of desired recommendations per cluster"""
    
    # Get all non_input_songs
    non_input_songs = all_songs.drop(input_songs.index, axis=0)
    #non_input_songs = all_songs
    
    # Fit clusters and get centroids
    kmeans = KMeans(n_clusters=nb_clusters).fit(input_songs[np.arange(20)])
    centroids = kmeans.cluster_centers_

    
    # Save recommedations
    recommendations = pd.DataFrame()

    for cluster_n, centroid in enumerate(centroids):
        # Save distance to current centroid in column "dist"
        non_input_songs.loc[:, 'dist'] = non_input_songs.apply(lambda row: get_dist(centroid, row.loc[np.arange(20)].values), axis=1)

        # Sort non input songs by distance to centroid and keep closest nb_rec_per_cluster
        cluster_rec = non_input_songs.sort_values(by='dist', axis=0)
        cluster_rec = cluster_rec.iloc[:nb_rec_per_cluster]
        cluster_rec.loc[:, 'cluster'] = cluster_n
        recommendations = recommendations.append(cluster_rec)
        
    return recommendations

#Evaluation
## Average mean average precision

In [0]:
def get_mean_average_precision(recommendations, left_outs):
    nb_clusters = recommendations.loc[:, 'cluster'].max(axis=0)+1
    nb_left_outs = left_outs.shape[0]
    sum_average_precision = 0
    
    for cluster in range(nb_clusters):
        sum_precision = 0
        cluster_recommendations = recommendations.loc[recommendations.cluster==cluster]
        nb_retrieved = 0 # true positives + false positives
        nb_correct = 0 # true positives
        for index, song in cluster_recommendations.iterrows():
            nb_retrieved += 1
            
            if index in left_outs.index:
                print(song.title, 'at rank', nb_retrieved)
                nb_correct += 1
                precision = nb_correct/nb_retrieved
                sum_precision += precision
        average_precision = sum_precision/nb_left_outs
        sum_average_precision += average_precision
    mean_average_precision = sum_average_precision/nb_clusters
    return mean_average_precision

In [0]:
# Model parameters
profile_length = 1000
nb_genres = 2
nb_rec_per_cluster = 1000

# Compute Average mean average precision over 10 user profiles
test_data = get_test_data()
sum_MAP = 0
for n in range(10):
    profile, tags = get_profile(test_data, length=profile_length, min_nb_genres=nb_genres)

    # Split profile into input and left_out songs
    nb_input_songs = int(np.rint(profile.shape[0]*0.9))
    input_songs = profile.iloc[:nb_input_songs]
    left_out_songs = profile.iloc[nb_input_songs:]

    # The number of tags is usually not given. In application, we would have to 
    # estimate this.
    nb_clusters = len(tags)
    recommendations = get_recommendations(test_data, input_songs, nb_clusters, nb_rec_per_cluster=nb_rec_per_cluster)

    profile_MAP = get_mean_average_precision(recommendations, left_out_songs)
    sum_MAP += profile_MAP
overall_MAP = sum_MAP/10
print(overall_MAP)

In [0]:
# Compute average precision of random recommendations
sum_average_precision = 0
for n in range(1000):
    # Get 100 random songs as recommendations
    indices = np.random.randint(0, test_data.shape[0], size=1000)
    random_rec = test_data.iloc[indices,:]
    # Compute random AP
    nb_left_outs = left_out_songs.shape[0] 
    sum_precision = 0
    nb_retrieved = 0 # true positives + false positives
    nb_correct = 0 # true positives
    for index, song in random_rec.iterrows():
        nb_retrieved += 1
        if index in left_out_songs.index:
            nb_correct += 1
            precision = nb_correct/nb_retrieved
            sum_precision += precision
    average_precision = sum_precision/nb_left_outs
    sum_average_precision += average_precision
    # Compare
randomMAP = sum_average_precision/1000
print(randomMAP) 


0.28870120637107916
